## Laboratorium 4
#### Bartosz Hanc

In [2]:
"""Requirements:
numpy==1.23.5
scikit-learn==1.2.1
scipy==1.10.0
kmedoids==0.4.3
"""

import numpy as np


1. Zaimplementuj przynajmniej 3 "metryki" spośród wymienionych: cosinusowa, LCS,
   DICE, euklidesowa, Levenshteina.

Ponieważ do klasteryzacji użyłem algorytmu PAM k-medoids
(https://en.wikipedia.org/wiki/K-medoids) dostępnego w bibliotece `kmedoids`,
którego implementacja zawiera krok przypisania punktu do klastra wyznaczonego
przez *najbliższy* medoid, więc wykorzystana funkcja podobieństwa wyrazów
powinna mieć własności standardowej metryki tj. $d(x,y) = 0\iff x=y$, $\forall
x,y: d(x,y) \geq 0$ i odpowiadać intuicyjnemu postrzeganiu, iż jeśli $d(x,y) <
d(x,z)$ to $y$ jest bliżej $x$, czyli wyraz $y$ jest bardziej podobny do $x$ niż
wyraz $z$. Spośród wymienionych funkcji jedynie odległość euklidesowa i
Levenshteina spełniają te kryteria. Z tego względu jako trzecią metrykę wybrałem
metrykę taksówkową spoza listy.

In [3]:
from sklearn.metrics import pairwise_distances as d

# Use cases:
# d(X, Y, metric="euclidean")
# d(X, Y, metric="manhattan")
# d(X, Y, metric="hamming") # This one is very slow because it does not support
# sparse matrices (and our matrix has ~1e8 elements)


2. Zaimplementuj przynajmniej 1 sposoby oceny jakości klasteryzacji (np. indeks
   Daviesa-Bouldina).

In [4]:
from sklearn.metrics import davies_bouldin_score as DB_idx


3. Stwórz stoplistę najczęściej występujących słów i zastosuj ją jako
   pre-processing dla nazw. Algorytmy klasteryzacji powinny działać na dwóch
   wariantach: z pre-processingiem i bez pre-processingu.

In [5]:
docs = np.array(open("lines.txt", "r").readlines())

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
fv = cv.fit_transform(docs)

stop_words = []
threshold = 400

for freq, word in zip(np.sum(fv.toarray(), axis=0), cv.get_feature_names_out()):
    if freq > threshold:
        stop_words.append(word)


4. Wykonaj klasteryzację zawartości załączonego pliku (lines.txt) przy użyciu
   metryk zaimplementowanych w pkt. 1. Każda linia to adres pocztowy firmy,
   różne sposoby zapisu tego samego adresu powinny się znaleźć w jednym
   klastrze.

In [15]:
import kmedoids

freq_vecs = CountVectorizer().fit_transform(docs)
freq_vecs_prep = CountVectorizer(stop_words=stop_words).fit_transform(docs)

k = 1000

c_euclid = kmedoids.fasterpam(d(freq_vecs, freq_vecs, metric="euclidean"), k)
c_manhat = kmedoids.fasterpam(d(freq_vecs, freq_vecs, metric="manhattan"), k)
c_hammin = kmedoids.fasterpam(
    d(freq_vecs.toarray(), freq_vecs.toarray(), metric="hamming", n_jobs=-1), k
)

c_euclid_prep = kmedoids.fasterpam(
    d(freq_vecs_prep, freq_vecs_prep, metric="euclidean"), k
)
c_manhat_prep = kmedoids.fasterpam(
    d(freq_vecs_prep, freq_vecs_prep, metric="manhattan"), k
)
c_hammin_prep = kmedoids.fasterpam(
    d(freq_vecs_prep.toarray(), freq_vecs_prep.toarray(), metric="hamming", n_jobs=-1),
    k,
)


In [16]:
def show_clusters(labels, docs, n=40, save_to_file=None):
    """Prints text clustering

    Args:
        labels (list[Any]): list of cluster labels

        docs (list[str]): list of texts

        n (int, optional): Prints last n lines. Defaults to 40.

        save_to_file (Any, optional): If type==str saves output to file 'save_to_file'. Defaults to None.
    """

    cluster_dict = {label: [] for label in labels}
    for label, text in zip(labels, docs):
        cluster_dict[label].append(text)

    s = "==" * 42 + "\n"
    for label, texts in cluster_dict.items():
        s += "#" * 42 + "\n"
        for text in texts:
            s += text
        s += "\n"
        
    s += "==" * 42 + "\n"

    if type(save_to_file) == str:
        open(save_to_file, "w").write(s)

    print("\n".join(s.split(sep="\n")[-n:]))


In [17]:
show_clusters(c_euclid.labels, docs)


ZHANGJIAGANG KAIDI PACKING CO.,LTD.GANGKOU INDUSTRIAL AREA OF FENGHUANG TOWN ZHANGJIAGANG CITY JIANGSU CHINA

##########################################
ZHANGJIAGANG TOOL-SUPER MANUFACTURECO.,LTD. DAXIN INDUSTRIAL ZONE , ZHANGJIAGAN CITY,JIANGSU PROVINCE,CHINA. PH +86-512-58938210 FAX:+86-512-589

##########################################
ZHEJIANG BADA HARDWARE CO.,LTD ADD:NO.20,QILI INDUSTRIAL ZONE, JINYUNCITY,ZHEJIANG PROVINCE,CHINA TEL:0086-578-3124789 FAX:0086-578-3124178+
ZHEJIANG BADA HARDWARE CO.,LTD NO.20,QILI INDUSTRIAL ZONE,JINYUN CITY,ZHEJIANG,CHINA 3214000 TEL:0086-578-3124789 FAX:0086-578-3124178

##########################################
ZHEJIANG DINGSHENG INTERNATIONAL  FORWARDER CO.,LTD NO.598  JINSHAN ROAD, CUSTOMS  BUILDING   SECOND FLOOR,YONGKANG,ZHEJIANG

##########################################
ZHEJIANG JONWAYMACHINERY&ELECTRICMANUFACTURE CO LTD.DATANG INDUSTRIALZONE JIANTIAO TOWN SANMEN COUNTYTAIZHOU CITY
ZHEJIANG JONWAYMACHINERY&ELECTRICMANUFACTURE CO LTD.DAT

In [18]:
show_clusters(c_manhat.labels, docs)


ZEN CONTINENTAL (TIANJIN) ENTERPRISES CO.,LTD.SHENZHEN BRANCH RM.D,12F, TIMES PLAZA, NO.1 TAIZI ROAD,SHEKOU,SHENZHEN,GUANGDONG, CHINA 0755-2681-8599 0755-2689-1817 ZIP:518067
ZEN CONTINENTAL (TIANJIN) ENTERPRISES CO.,LTD.SHENZHEN BRANCH RM.D,12F, TIMES PLAZA, NO.1 TAIZI ROAD,SHEKOU,SHENZHEN,GUANGDONG, CHINA 0755-2681-8599 0755-2689-1817  ZIP:518067
ZEN CONTINENTAL (TIANJIN) ENTERPRISES CO.,LTD.SHENZHEN BRANCH RM.D,12F, TIMES PLAZA, NO.1 TAIZI ROAD,SHEKOU, SHENZHEN,GUANGDONG, CHINA 0755-2681-8599 0755-2689-1817 ZIP:518067
ZEN CONTINENTAL (TIANJIN) ENTERPRISES CO.,LTD.SHENZHEN BRANCH RM.D,12F, TIMES PLAZA, NO.1 TAIZI ROAD, SHEKOU,SHENZHEN,GUANGDONG, CHINA 0755-2681-8599 0755-2689-1817 ZIP:518067

##########################################
ZHANGJIAGANG LOOP IMP.AND EXP CORP15E FUGANG BLDG.,RENMIN ROAD ZHANGJIAGANG CITY,JIANGSU PROVINCE,CN
ZHANGJIAGANG LOOP IMP.AND EXP CORP15E FUGANG BLDG., RENMIN ROAD ZHANGJIAGANG CITY,JIANGSU PROVINCE,CN

##########################################
ZHEJIA

In [19]:
show_clusters(c_hammin.labels, docs)


ZHANGZHOU HUAYUAN TRADING CO.,LTD.ADD:3#-404,ZHONG XING AI XIAN GARDEN,SOUTH YUAN GUANG ROAD, ZHANGZHOUFUJIAN CHINA

##########################################
ZHEJIANG BADA HARDWARE CO.,LTD ADD:NO.20,QILI INDUSTRIAL ZONE, JINYUNCITY,ZHEJIANG PROVINCE,CHINA TEL:0086-578-3124789 FAX:0086-578-3124178+

##########################################
ZHEJIANG BADA HARDWARE CO.,LTD NO.20,QILI INDUSTRIAL ZONE,JINYUN CITY,ZHEJIANG,CHINA 3214000 TEL:0086-578-3124789 FAX:0086-578-3124178

##########################################
ZHEJIANG CHAIRMAN FURNITURE CO.,LTDYANGGUANG INDUSTRIAL DEVELOPMENTZONE,ANJI COUNTY,ZHEJIANG,CHINA
ZHEJIANG CHAIRMAN FURNITURE  CO.,LTD YANGGUANG INDUSTRIAL  DEVELOPMENTZONE,ANJI COUNTY,  ZHEJIANG,CHINA

##########################################
ZHEJIANG CONCORD SHEET STEEL. SCIENCE AND TECHNOLOGY CO.,LTD. HONG SHAN ROAD,XIAO SHAN DISTRICT, HANG ZHOU CITY,ZHEJIANG,CHINA TEL:571-86-571-82699888 FAX:86-571-82622802 O/B:

##########################################
ZHEJIANG 

In [20]:
show_clusters(c_euclid_prep.labels, docs)

ZEN CONTINENTAL CO INC (SZH OFFICE)TAIZI RD SHEKOU 27D HAIJING QUARE518000 SHENZHEN CHINA SHENZHEN 44 CN

##########################################
ZHEJIANG BADA HARDWARE CO.,LTD ADD:NO.20,QILI INDUSTRIAL ZONE, JINYUNCITY,ZHEJIANG PROVINCE,CHINA TEL:0086-578-3124789 FAX:0086-578-3124178+
ZHEJIANG BADA HARDWARE CO.,LTD NO.20,QILI INDUSTRIAL ZONE,JINYUN CITY,ZHEJIANG,CHINA 3214000 TEL:0086-578-3124789 FAX:0086-578-3124178
ZHEJIANG FLYHIGH METAL PRODUCTS CO., LTD JINYUN INDUSTRIAL ZONE,JINYUNCITY, ZHEJIANG,P.R.CHINA
ZHEJIANG GUANGYING MACHINERY CO., LTD. 321403,NO.12, XINZHENG ROAD ,JINYUN INDUSTRIAL ZONE, ZHEJIANG, CHINA
ZHEJIANG RONGRONG INDUSTRIAL CO.,LTD HUANGLONG INDUSTRIAL ZONE,WUYI,ZHEJIANG,CHINA
ZHEJIANG WUYI GUANXIN MACHINERY CO.,LTD JIN YAN SHAN INDUSTRIAL ZONE QUANXI TOWN,WUYI COUNTY,ZHEJIANG PROVINCE,CHINA
ZHEJIANG ZHUJI HORIZON CLOTHING INDUSTRIAL CO., LTD NO 20, CHANGPING ROAD, CHENGXI INDUSTRIAL ZONE,ZHUJICITY, ZHEJIANG PROVINCE,CHINA

#####################################

In [21]:
show_clusters(c_manhat_prep.labels, docs)


##########################################
ZHANGJIAGANG LOOP IMP.AND EXP CORP15E FUGANG BLDG.,RENMIN ROAD ZHANGJIAGANG CITY,JIANGSU PROVINCE,CN
ZHANGJIAGANG LOOP IMP.AND EXP CORP15E FUGANG BLDG., RENMIN ROAD ZHANGJIAGANG CITY,JIANGSU PROVINCE,CN

##########################################
ZHEJIANG AOTAI MACHINE MANUFACTURING CO.,LTD THE 10TH ROAD ,BINHAI THIRD AVNUE,BINHAI INDUSTRY AREA,WENZHOU,325025,CHINA CONTACT:PIER WANG TEL:+86-577-86830918 FAX:+86 577-868

##########################################
ZHEJIANG BADA HARDWARE CO.,LTD ADD:NO.20,QILI INDUSTRIAL ZONE, JINYUNCITY,ZHEJIANG PROVINCE,CHINA TEL:0086-578-3124789 FAX:0086-578-3124178+
ZHEJIANG BADA HARDWARE CO.,LTD NO.20,QILI INDUSTRIAL ZONE,JINYUN CITY,ZHEJIANG,CHINA 3214000 TEL:0086-578-3124789 FAX:0086-578-3124178

##########################################
ZHEJIANG CONCORD SHEET STEEL. SCIENCE AND TECHNOLOGY CO.,LTD. HONG SHAN ROAD,XIAO SHAN DISTRICT, HANG ZHOU CITY,ZHEJIANG,CHINA TEL:571-86-571-82699888 FAX:86-571-8262280

In [22]:
show_clusters(c_hammin_prep.labels, docs)

##########################################
ZHEJIANG ANJI RONGYI FURNITURE CO.,LTD TANGPU INDUSTRY ZONE, ANJI COUNTY ZHEJIANG PROVINCE CHINA
ZHEJIANG AOTE TOOLS CO.,LTD PHOENIXINDUSTRIAL ZONE,WUYI COUNTY, ZHEJIANG PROVINCE, P.R.CHINA
ZHEJIANG HENGLIN CHAIR INDUSTRY CO.,LTD 3BLOCK SUNLIGHT INDUSTRY  ZONE,ANJI COUNTY,ZHEJIANG PROVINCE, CHINA
ZHEJIANG UE FURNITURE CO., LTD. NO.1 YONGYI WEST ROAD,ANJI COUNTY ZHEJIANG PROVINCE,CHINA

##########################################
ZHEJIANG CHAOYUE TRADING COMPANY LTD. NO.69 SOUTH WANGJIANG ROAD, DONGYANG CITY ZHEJIANG CHINA

##########################################
ZHEJIANG GUANGYING MACHINERY CO., LTD. 321403,NO.12, XINZHENG ROAD ,JINYUN INDUSTRIAL ZONE, ZHEJIANG, CHINA

##########################################
ZHEJIANG GUXIANDAO INDUSTRIAL FIBRECO., LTD., YUEDONG ROAD PAOJIANG INDUSTRIAL DISTRICT SHAOXING 312000CHINA

##########################################
ZHEJIANG HUATAI INTL FORWARDING COVDR165 ZHONGHE ZHONG ROAD310000 HANGZHOU CHINA


5. Porównaj jakość wyników sposobami zaimplementowanymi w pkt. 2.

In [24]:
lines = open("clusters.txt", "r").read().splitlines()
lines = list(filter(lambda text: text != "", lines))

model_docs, model_labels, label = [], [], 0
for line in lines:
    if line != "#" * 10:
        model_docs.append(line)
        model_labels.append(label)
    else:
        label += 1

model = CountVectorizer().fit_transform(model_docs)

In [27]:
print("Davies--Bouldin indexes\n" + "_" * 42 + "\n")
print(
    "Model clusterization                ", DB_idx(model.toarray(), labels=model_labels)
)

print("\nWITHOUT PREPROCESSING")
print(
    "k-medoids clusterization (euclidean)",
    DB_idx(freq_vecs.toarray(), c_euclid.labels),
)
print(
    "k-medoids clusterization (manhattan)",
    DB_idx(freq_vecs.toarray(), c_manhat.labels),
)
print(
    "k-medoids clusterization (hamming)  ",
    DB_idx(freq_vecs.toarray(), c_hammin.labels),
)

print("\nWITH PREPROCESSING")
print(
    "k-medoids clusterization (euclidean)",
    DB_idx(freq_vecs_prep.toarray(), c_euclid_prep.labels),
)
print(
    "k-medoids clusterization (manhattan)",
    DB_idx(freq_vecs_prep.toarray(), c_manhat_prep.labels),
)
print(
    "k-medoids clusterization (hamming)  ",
    DB_idx(freq_vecs_prep.toarray(), c_hammin_prep.labels),
)


Davies--Bouldin indexes
__________________________________________

Model clusterization                 1.112534158155742

WITHOUT PREPROCESSING
k-medoids clusterization (euclidean) 1.4929700218844801
k-medoids clusterization (manhattan) 1.3532903921466326
k-medoids clusterization (hamming)   1.5570908383441069

WITH PREPROCESSING
k-medoids clusterization (euclidean) 1.4698938464183584
k-medoids clusterization (manhattan) 1.3186055425104957
k-medoids clusterization (hamming)   1.3535581942929484


Na podstawie uzyskanych wyników możemy stwierdzić, iż spośród testowanych metryk
najlepszą klasteryzację (tj. o najniższym indeksie DB) uzyskano dla metryki
taksówkowej. Zastosowanie stoplisty najczęstszych słów polepsza klasteryzację.

6. Czy masz jakiś pomysł na poprawę jakości klasteryzacji w tym zadaniu?

Jakość klasteryzacji w przypadku użytego algorytmu zależy od parametru $k$
określającego liczbę klastrów, która musi zostać podana a priori. W powyższym
przykładzie liczbę klastrów można oszacować mając modelową klasteryzację, ale w
ogólności jej nie znamy. Aby poprawić jakość klasteryzacji, można zatem znaleźć
liczbę k, która minimalizuje pewną funkcję (np. indeks DB) określającą jakość.
Ze względu na fakt, iż ewaluacja takiej funkcji jest kosztowna (za każdym razem
wykonujemy klasteryzację dla innego k) korzystne mogłoby być wykorzystanie
teorii procesów Gaussowskich do tego celu. Innym parametrem, który podajemy a
priori, jest minimalna liczba wystąpień danego słowa, aby zaklasyfikować je do
stoplisty. Analogicznie można więc minimalizować indeks DB ze względu na ten
parametr, aby uzyskać lepszą klasteryzację.